In [2]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer

operations_ids = []


prev_titles = []
news_dataset = pd.read_csv('Lenta_dataset.csv')
news_dataset.head()

,Unnamed: 0,url,title,text,topic,tags,date
0,0,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир,NaN
1,1,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика,NaN
2,2,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество,NaN
3,3,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина,NaN
4,4,https://lenta.ru/news/2018/12/15/skrepy/,Российскую молодежь предложили обучать духовны...,Российская молодежь лучше усвоит духовные ценн...,Интернет и СМИ,Coцсети,NaN


In [3]:
safe_df = news_dataset[~news_dataset['tags'].isin(['Все','Политика', 'Первая мировая', 'Россия', 'Вооружение', 'Выборы', 'Киберпреступность','Украина', 'Молдавия', 'Преступная Россия', 'Полиция и спецслужбы', 'Конфликты', 'Преступность', 'Криминал', 'Оружие', 'Следствие и суд'])]
generated_result = pd.DataFrame(columns=['title', 'generated_text'])

In [24]:
now_titles = safe_df.title.sample(600)
titles =  [x for x in now_titles if x not in prev_titles]
prev_titles = prev_titles + titles

In [11]:
model_name="ai-forever/ruGPT-3.5-13B"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
def generate_text(prompt, model, tokenizer,  max_length=1000):
    encoded_input = tokenizer.encode(prompt, return_tensors='pt')
    
    output_sequences = model.generate(
        input_ids=encoded_input,
        max_length=max_length,
        temperature=0.7,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1
    )

    text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    end_pos = text.rfind('.')
    if end_pos != -1:
        text = text[:end_pos+1]
    return text

generated = []
for title in titles[:]:
    prompt_text = f"Новостная статья: {title}"
    generated_text = generate_text(prompt_text, model, tokenizer, max_length=1000)
    generated.append((title, generated_text))



In [26]:
tmp = pd.DataFrame(generated, columns=['title','generated_text'])
generated_result = pd.concat([generated_result, tmp])
generated_result.drop_duplicates().reset_index(drop=True)

,title,generated_text
0,Раскрыты премии красноармейцев за бомбардировк...,Новостная статья: Раскрыты премии красноармейц...
1,Безлимитный мобильный интернет уходит с рынка,Новостная статья: Безлимитный мобильный интерн...
2,Опоссум-победитель стал героем «Звездных войн»...,Новостная статья: Опоссум-победитель стал геро...
3,Общественный совет Минкульта снова заявил о не...,Новостная статья: Общественный совет Минкульта...
4,Petit Bateau открыл российский интернет-магазин,Новостная статья: Petit Bateau открыл российск...
...,...,...
218,Клуб шестого дивизиона Германии потребовал у «...,Новостная статья: Клуб шестого дивизиона Герма...
219,К Национальной платежной системе присоединилас...,Новостная статья: К Национальной платежной сис...
220,Блогер выучил русский мат перед «холодной войн...,Новостная статья: Блогер выучил русский мат пе...
221,Госдума отложила реформу газового налога на по...,Новостная статья: Госдума отложила реформу газ...


In [27]:
generated_result.to_csv('generated_rugpt.csv')